In [1]:
#!pip install ray[train]

In [2]:
#!pip install -U ipywidgets
# not needed for production, only for jupyter notebook

In [1]:
#--- should clean imports - copy paste
import ray
from ray.train import ScalingConfig
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor, Normalize, Compose
import ray.train.torch
import ray.util.state
import ray
import torch
import torch.nn as nn
import ray.train.torch
import ray.train
from torch.nn import CrossEntropyLoss, MSELoss
from torch.optim import Adam

In [ ]:
ray.init()

In [10]:
from quickannotator.dl.training import train_model

#@ray.remote
class DLActor:
    def __init__(self,classid,status=-1):
        self.classid=classid
        self.status=status
        
    def start_dlproc(self,is_train=True,allow_pred=True):
        #need to check if is already training, if yes, no opt
        print("starting up")
        
        
        
        #scaling_config = ray.train.ScalingConfig(num_workers=2, use_gpu=True,resources_per_worker={"GPU":.5})
        ## will fail on single node single gpu --- (DLActor pid=174521) Duplicate GPU detected : rank 1 and rank 0 both on CUDA device 1000

        #--- NEED A GPU CONTAINER TO TEST THIS        
        # total_gpus  = ray.cluster_resources().get("GPU", 0)
        # total_gpus_available=  ray.available_resources().get("GPU", 0)
        # print(total_gpus,total_gpus_available)
        # resource_to_request = (total_gpus_available/total_gpus)/2 * 1.01
        # print(f"resource_to_request: {resource_to_request}")

        # scaling_config = ray.train.ScalingConfig(num_workers=total_gpus, use_gpu=True, resources_per_worker={"GPU":resource_to_request})
        

        #----
        scaling_config = ray.train.ScalingConfig(num_workers=1, use_gpu=False)
        trainer = ray.train.torch.TorchTrainer(train_model,
                                       scaling_config=scaling_config,
                                       train_loop_config={'is_train':is_train, 
                                                          'allow_pred':allow_pred,
                                                         'classid':self.classid})
        self.status = trainer.fit().hex() ##widly -- this doesn't save the result in the actor
        return self.status
    
    # def infer(self,tileid):
    #     import datetime
    #     #go into seen database, set tileid equal to true, and set date
    #     Seen,session = setupdb()
        

    #     row_to_update = session.query(Seen).filter(Seen.tileid == tileid,
    #                                                Seen.classid==self.classid).first()

    #     if row_to_update:
            
    #         row_to_update.date = datetime.datetime.now()
    #         row_to_update.status = 1

    #         # Commit the changes
    #         session.commit()
    #         print(f"Updated tileid {tileid}: status set to 1 and date updated.")
    #     else:
    #         print(f"No entry found for tileid {tileid}.")

    #     session.close()

    #     #if start_dlproc not running, then need to start it, but set it to no train, just pred --- start_dlproc(is_train=False)
        
    #     return tileid
        
        

    # def bulkinfer(self,tileids):
    #     import sqlalchemy, datetime
    #     Seen,session = setupdb()
        

    #     stmt = (sqlalchemy.update(Seen).where(Seen.tileid.in_(tileids),
    #                                           Seen.classid==self.classid).values(status=1,date=datetime.datetime.now()))

    #     # Execute the update
    #     session.execute(stmt)
    #     session.commit()
        
    #     # Close the session
    #     session.close()


    #     #need a similar statement to get the DL starting
    #     return True
        
    def getclassid(self):
        return self.classid
        
    
    def getstatus(self):
        print("in get status!")
        print(f"self.status:\t {self.status}")
        return self.status

    def setstatus(self,status):
        self.status=status
        return self.status



In [11]:
class2=DLActor(classid=2)

In [ ]:
class2.start_dlproc(is_train=True,allow_pred=True)

In [ ]:
dd

In [ ]:
class2 = DLActor.options(max_concurrency=2).remote(classid=2)


In [ ]:
print(ray.get(class2.getclassid.remote()))


In [ ]:
ref2=class2.start_dlproc.remote()
print(ref2)

In [18]:
%load_ext autoreload

In [19]:
%autoreload 2

In [20]:
from quickannotator.dl.training import train_model

In [33]:
from quickannotator.db import db, Project, Image, AnnotationClass, Notification, Tile, Setting, Annotation, SearchCache


In [ ]:
#--- need to run 
#memcached -m 10240 -I 20m -vv -u www-data &

In [49]:
train_model({"classid":2,"tile_size":2_048})

/opt/QuickAnnotator/quickannotator/dl/training.py:26: UserWarning: Argument 'eps' is not valid and will be ignored.
  A.RandomGamma(p=.5, gamma_limit=(80, 120), eps=1e-07),
InitSpatiaMetaData() error:"table spatial_ref_sys already exists"


len(self.tiles)=5499


0it [00:00, ?it/s]InitSpatiaMetaData() error:"table spatial_ref_sys already exists"
InitSpatiaMetaData() error:"table spatial_ref_sys already exists"
3it [00:42, 14.26s/it]


KeyboardInterrupt: 